## LLM - Homework 01-intro

### Q1. Running Elastic

Rta1: build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7

### Getting the data

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2. Indexing the data

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es_client = Elasticsearch('http://localhost:9200')

In [4]:
es_client.info()

ObjectApiResponse({'name': 'c20f7c4c7d53', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'kDPMBhCAQcerKOMz1DMyzw', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

Rta2: index

### Q3. Searching

In [7]:
from tqdm.auto import tqdm

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [23]:
query = 'How do execute a command on a Kubernetes pod?'

In [29]:
def elastic_search(query):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                   }
                }
            }
        }
    

    response = es_client.search(index=index_name, body=search_query)
    
    return response

In [31]:
elastic_search(query)['hits']['hits'][0]['_score']

44.50556

Rta3: 44.50

In [20]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [21]:
query = 'I just discovered the course. Can I still join it?'

In [22]:
rag(query)

NameError: name 'client' is not defined